# DSSATTools Example notebook

In this notebook you'll see the basic usage of DSSATools library to:
1. Create the inputs
2. Initialize the simulation environment
3. Run simulations
4. Modify input objects
5. Read outputs.

The idea of this library is to be a low-code interface to DSSAT

In [1]:
from DSSATTools import (
    Crop, SoilProfile, WeatherData, WeatherStation,
    Management, DSSAT, TabularSubsection
)
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [7]:
# Random weather data
DATES = pd.date_range('2000-01-01', '2002-12-31')
N = len(DATES)
df = pd.DataFrame(
    {
    'tn': np.random.gamma(10, 1, N),
    'rad': np.random.gamma(10, 1.5, N),
    'prec': [0.0]* N,
    'rh': 100 * np.random.beta(1.5, 1.15, N),
    },
    index=DATES,
)
df['TMAX'] = df.tn + np.random.gamma(5., .5, N)

## Creating Weather, Soil and Crop instances

In [8]:
# Create a WeatherData instance
WTH_DATA = WeatherData(
    df,
    variables={
        'tn': 'TMIN', 'TMAX': 'TMAX',
        'prec': 'RAIN', 'rad': 'SRAD',
        'rh': 'RHUM'
    }
)
# Create a WheaterStation instance
wth = WeatherStation(
    WTH_DATA, 
    {'ELEV': 33, 'LAT': 0, 'LON': 0, 'INSI': 'dpoes'}
)

# Soil instance from default soil profile
soil = SoilProfile(default_class='SIC')

# Crop
crop = Crop('Potato')

## Defining irrigation schedule

In [30]:
schedule = pd.DataFrame([
    (datetime(2000,1,15), 80),
    (datetime(2000,1,30), 60),
    (datetime(2000,2,15), 40),
    (datetime(2000,3,1),  20)
], columns = ['date', 'IRVAL'])
schedule['IDATE'] = schedule.date.dt.strftime('%y%j')
schedule['IROP'] = 'IR001' # Code to define the irrigation operation
schedule = schedule[['IDATE', 'IROP', 'IRVAL']]
schedule

,IDATE,IROP,IRVAL
0,00015,IR001,80
1,00030,IR001,60
2,00046,IR001,40
3,00061,IR001,20


## Create a management instance

In [31]:
# Management instance
man = Management(
    cultivar='IB0001',
    planting_date=DATES[10],
)
# Modify harvest to Maturity
man.simulation_controls['HARVS'] = 'M'
# Set the irrigation schedule
man.irrigation['table'] = TabularSubsection(schedule)
# Set irrigation control as reported schedule
man.simulation_controls['IRRIG'] = 'R'

# The definition of this parameters is mandatory for roots
man.planting_details['table']['PLWT'] = 1500
man.planting_details['table']['SPRL'] = 2

In [32]:
dssat = DSSAT()
dssat.setup()
dssat.run(
    soil=soil, weather=wth, crop=crop, management=man,
)
# Run and check the final Yield

/tmp/dssatbijrzino created.
Static files copied to /tmp/dssatbijrzino.
RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 PT   1  29 -99 10974  9268     0   200   328     0     0   -99      0  206


In [33]:
# Modify the irrigation amounts and check output
schedule['IRVAL'] = [20, 20, 20, 20]
man.irrigation['table'] = TabularSubsection(schedule)
dssat.run(
    soil=soil, weather=wth, crop=crop, management=man,
)

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 PT   1  28 -99  3049  2295     0    80   205     0     0   -99      0  206


In [34]:
# Terminate the DSSAT instance
dssat.close()

/tmp/dssatbijrzino and its content has been removed.
